# Module 3: Data Transformations with Snowpark

## Start a Snowpark session with:
#####       `get_active_session()`:

In [ ]:
from snowflake.snowpark.functions import col, max as sp_max, year, month


# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


## Load the daily_weather_v using session.table

In [ ]:
daily_weather = session.table("tasty_bytes.harmonized.daily_weather_v")

filtered_weather = daily_weather.filter(
    (col("country_desc") == "Germany") &
    (col("city_name") == "Hamburg") &
    (year(col("date_valid_std")) == 2022) &
    (month(col("date_valid_std")) == 2)
)

## Perform filtering, aggregations, and sorting

In [ ]:
aggregated_weather = filtered_weather.groupBy(
    "country_desc", "city_name", "date_valid_std"
).agg(
    sp_max("max_wind_speed_100m_mph").alias("max_wind_speed_100m_mph")
)

sorted_weather = aggregated_weather.sort(col("date_valid_std").desc())


## Display the results

In [ ]:
sorted_weather.show(30)

## We can directly produce the output through sql

In [ ]:
sorted_weather = session.sql("""
    SELECT
        country_desc,
        city_name,
        date_valid_std,
        MAX(max_wind_speed_100m_mph) AS max_wind_speed_100m_mph
    FROM tasty_bytes.harmonized.daily_weather_v
    WHERE country_desc = 'Germany'
      AND city_name = 'Hamburg'
      AND YEAR(date_valid_std) = 2022
      AND MONTH(date_valid_std) = 2
    GROUP BY
        country_desc,
        city_name,
        date_valid_std
    ORDER BY date_valid_std DESC
""")

In [ ]:
sorted_weather.show(10)

In [ ]:
sorted_weather.create_or_replace_view("tasty_bytes.harmonized.windspeed_hamburg_snowpark")

## Cross-check the recently created view

In [ ]:
select * from tasty_bytes.harmonized.windspeed_hamburg_snowpark limit 5

## Parameterized SQL

In [ ]:
sorted_weather = session.sql("""
    SELECT
        country_desc,
        city_name,
        date_valid_std,
        MAX(max_wind_speed_100m_mph) AS max_wind_speed_100m_mph
    FROM tasty_bytes.harmonized.daily_weather_v
    WHERE country_desc = ?
      AND city_name = ?
      AND YEAR(date_valid_std) = ?
      AND MONTH(date_valid_std) = ?
    GROUP BY
        country_desc,
        city_name,
        date_valid_std
    ORDER BY date_valid_std DESC
""", params=["Germany", "Hamburg", 2022, 2])


In [ ]:
sorted_weather.show(5)